In [ ]:
import scanpy as sc
import squidpy as sq
import numpy as np
import pandas as pd
from anndata import AnnData
import pathlib
import matplotlib.pyplot as plt
import matplotlib as mpl
import skimage
import seaborn as sns
import tangram as tg
import geopandas as gpd
from shapely.geometry import Polygon
import muon as mu

sc.logging.print_header()
print(f"squidpy=={sq.__version__}")

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
def get_centroid(row):
    all_x = []
    all_y = []
    # Collect boundary coordinates, handle NaN gracefully
    for i in range(7):  # Adjust based on the number of slices
        x_col = f"boundaryX_z{i}"
        y_col = f"boundaryY_z{i}"

        if pd.notna(row[x_col]) and pd.notna(row[y_col]):
            # Ensure we are extracting valid data
            x_values = list(map(float, str(row[x_col]).split(',')))

            y_values = list(map(float, str(row[y_col]).split(',')))

            if len(x_values) == len(y_values):  # Ensure matching x/y pairs
                all_x.extend(x_values)
                all_y.extend(y_values)

    # Proceed only if enough coordinates are available
    if len(all_x) > 2 and len(all_y) > 2:  # Ensure enough points for a polygon
        polygon = Polygon(zip(all_x, all_y))
        return polygon.centroid.x, polygon.centroid.y
        #if polygon.is_valid and not polygon.is_empty:
         #   return polygon.centroid.x, polygon.centroid.y
    
    # If there are no valid polygon points, use a fallback (average of available points)
    if len(all_x) > 0 and len(all_y) > 0:
        return np.mean(all_x), np.mean(all_y)
    return np.nan, np.nan  # Return NaN if no valid coordinates

In [ ]:
# Load merged file
segmented_cells = pd.read_csv("data/spatial_data/MERFISH_MOp/processed_data/all_segmented_cells.csv")

segmented_cells = segmented_cells.reset_index().rename(columns={"index": "cell_id"})

adata_st = sc.read_h5ad("data/spatial_data/MERFISH_MOp/processed_data/counts.h5ad")

cell_labels = pd.read_csv("data/spatial_data/MERFISH_MOp/processed_data/cell_labels.csv", index_col=0)  # Use sample_id as index

# Ensure index matches adata_st.varbb
cell_labels = cell_labels.loc[adata_st.obs.index]  

# Add to AnnData
adata_st.obs = cell_labels
#adata_st.obs = adata_st.obs.reset_index().rename(columns={"index": "cell_id"})

In [ ]:
smaller_slicrs = slice_counts[slice_counts<=4234]

In [ ]:
slice_counts = segmented_cells["slice_id"].value_counts()

closest_slice = slice_counts.iloc[(slice_counts - 4234).argsort()[:1]]

closest_slices = slice_counts.iloc[(slice_counts - 4234).argsort()[0:35]]
larger_slices = slice_counts[slice_counts > 4234]
print(larger_slices)
print(len(larger_slices))

In [ ]:
import shutil
import os

In [ ]:
#### copy for smaller



# Ensure the output directory exists
output_dir = "figures/spatial/spatial_plots/smaller"
os.makedirs(output_dir, exist_ok=True)

# List of genes to plot
merfish_test_genes = ["Muc20", "Nr4a2", "Tcap", "Rxfp2", "Rxfp1", "Rspo1", "Sulf1", "Npas1", "Syt6"]

# Iterate over each slice in larger_slices
for slice_id in smaller_slicrs.index:
    print(f"Processing slice: {slice_id}")

    # Subset segmented cells and AnnData
    segmented_cells_selected = segmented_cells[segmented_cells["slice_id"] == slice_id]
    adata_subset = adata_st[adata_st.obs["slice_id"] == slice_id].copy()

    # Extract Nxph4 expression
    if "Nxph4" in adata_subset.var_names:
        nxph4_expr = adata_subset[:, "Nxph4"].X.toarray().flatten()
        threshold = np.percentile(nxph4_expr, 95)
        subcortical_cells = adata_subset.obs.index[nxph4_expr > threshold]
        adata_subset = adata_subset[~adata_subset.obs.index.isin(subcortical_cells)]

    # Compute centroids
    segmented_cells_selected["centroid_x"], segmented_cells_selected["centroid_y"] = zip(*segmented_cells_selected.apply(get_centroid, axis=1))

    # Assign cell IDs for merging
    segmented_cells_selected["cell_id"] = segmented_cells_selected["Unnamed: 0"].astype(str)
    adata_subset.obs["cell_id"] = adata_subset.obs.index.astype(str)

    # Merge centroids into obs
    adata_subset.obs = adata_subset.obs.merge(segmented_cells_selected[["cell_id", "centroid_x", "centroid_y"]], on="cell_id", how="left")

    # Assign spatial coordinates
    adata_subset.obsm["spatial"] = np.array(adata_subset.obs[["centroid_x", "centroid_y"]])

    # Save spatial embedding plot
    sc.pl.embedding(adata_subset, basis="spatial", color="subclass", save=f"_{slice_id}.png")
    shutil.move(f"figures/spatial_{slice_id}.png", f"{output_dir}/spatial_{slice_id}.png")

    # Check which genes are available in the dataset
    available_genes = [gene for gene in merfish_test_genes if gene in adata_subset.var_names]
    
    # Generate and save spatial gene expression plots
    for gene in available_genes:
        sc.pl.spatial(
            adata_subset, 
            color=gene, 
            title=f"{gene} ({slice_id})", 
            spot_size=30, 
            alpha=0.8, 
            cmap="viridis",
            vmax=1,
            save=f"_{gene}_{slice_id}.png"
        )
        shutil.move(f"figures/show_{gene}_{slice_id}.png", f"{output_dir}/{gene}_{slice_id}.png")

    print(f"Finished processing slice: {slice_id}\n")

print("All plots have been generated and saved.")


In [ ]:
# Ensure the output directory exists
output_dir = "figures/spatial/spatial_plots"
os.makedirs(output_dir, exist_ok=True)

# List of genes to plot
merfish_test_genes = ["Muc20", "Nr4a2", "Tcap", "Rxfp2", "Rxfp1", "Rspo1", "Sulf1", "Npas1", "Syt6"]

# Iterate over each slice in larger_slices
for slice_id in larger_slices.index:
    print(f"Processing slice: {slice_id}")

    # Subset segmented cells and AnnData
    segmented_cells_selected = segmented_cells[segmented_cells["slice_id"] == slice_id]
    adata_subset = adata_st[adata_st.obs["slice_id"] == slice_id].copy()

    # Extract Nxph4 expression
    if "Nxph4" in adata_subset.var_names:
        nxph4_expr = adata_subset[:, "Nxph4"].X.toarray().flatten()
        threshold = np.percentile(nxph4_expr, 95)
        subcortical_cells = adata_subset.obs.index[nxph4_expr > threshold]
        adata_subset = adata_subset[~adata_subset.obs.index.isin(subcortical_cells)]

    # Compute centroids
    segmented_cells_selected["centroid_x"], segmented_cells_selected["centroid_y"] = zip(*segmented_cells_selected.apply(get_centroid, axis=1))

    # Assign cell IDs for merging
    segmented_cells_selected["cell_id"] = segmented_cells_selected["Unnamed: 0"].astype(str)
    adata_subset.obs["cell_id"] = adata_subset.obs.index.astype(str)

    # Merge centroids into obs
    adata_subset.obs = adata_subset.obs.merge(segmented_cells_selected[["cell_id", "centroid_x", "centroid_y"]], on="cell_id", how="left")

    # Assign spatial coordinates
    adata_subset.obsm["spatial"] = np.array(adata_subset.obs[["centroid_x", "centroid_y"]])

    # Save spatial embedding plot
    sc.pl.embedding(adata_subset, basis="spatial", color="subclass", save=f"_{slice_id}.png")
    shutil.move(f"figures/spatial_{slice_id}.png", f"{output_dir}/spatial_{slice_id}.png")

    # Check which genes are available in the dataset
    available_genes = [gene for gene in merfish_test_genes if gene in adata_subset.var_names]
    
    # Generate and save spatial gene expression plots
    for gene in available_genes:
        sc.pl.spatial(
            adata_subset, 
            color=gene, 
            title=f"{gene} ({slice_id})", 
            spot_size=30, 
            alpha=0.8, 
            cmap="viridis",
            vmax=1,
            save=f"_{gene}_{slice_id}.png"
        )
        shutil.move(f"figures/show_{gene}_{slice_id}.png", f"{output_dir}/{gene}_{slice_id}.png")

    print(f"Finished processing slice: {slice_id}\n")

print("All plots have been generated and saved.")
